In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2023-03-05 16:31:09.951219


In [7]:
import urllib3, json
import pandas as pd 
http = urllib3.PoolManager() 
pd.set_option("display.max.columns", None) 
 
url = "https://litteraturbanken.se/api/get_authors"
r = http.request('GET', url)
data = json.loads(r.data)
dfLitt = pd.json_normalize(data["data"])

In [11]:
dfLittShowTrue = dfLitt[dfLitt["show"] == True].copy()
dfLittShowTrue.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3719 entries, 2 to 4873
Data columns (total 50 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   authorid                       3719 non-null   object 
 1   authorid_norm                  3719 non-null   object 
 2   db_checksum                    3719 non-null   object 
 3   db_timestamp                   3719 non-null   int64  
 4   doc_type                       3719 non-null   object 
 5   full_name                      3719 non-null   object 
 6   gender                         3719 non-null   object 
 7   imported                       3290 non-null   object 
 8   intro                          722 non-null    object 
 9   name_for_index                 3719 non-null   object 
 10  pictureinfo                    245 non-null    object 
 11  searchable                     3719 non-null   bool   
 12  show                           3719 non-null   b

In [12]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys,json
import pandas as pd 

from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

# 
query = """SELECT (REPLACE(STR(?item), ".*Q", "Q") AS ?WikidataID)  ?authorid ?SBL ?SKBL WHERE {
?item wdt:P31 wd:Q5.
?item wdt:P5101 ?authorid
OPTIONAL {?item wdt:P3217 ?SBL}
OPTIONAL {?item wdt:P4963 ?SKBL}
} order by ?authorid"""
 
    
def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']
    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

WDLittbanktot = get_sparql_dataframe(endpoint_url, query)

In [13]:
WDLittbanktot

,WikidataID,authorid,SBL,SKBL
0,Q11967131,AasenE,None,None
1,Q4933592,AbeniusM,None,MargitAbenius
2,Q24680938,AbrahamssonA,None,None
3,Q4934135,AbrahamssonS,None,None
4,Q365923,AchariusE,5503,None
...,...,...,...,...
3349,Q108406266,ÖstergrenC,None,None
3350,Q6257795,ÖstergrenCL,None,None
3351,Q100752816,ÖstinO,None,None
3352,Q6258216,ÖstmanK,None,None


In [20]:
# df = pd.merge(dfA, dfB, on=['a','b'], how="outer", indicator=True)
WDLittbank_WD_merge=  pd.merge(dfLittShowTrue,WDLittbanktot, how="outer", on='authorid',indicator=True)
WDLittbank_WD_merge.rename(columns={"_merge": "WD_Littbank_merge_Outer"},inplace = True)

WDLittbank_WD_merge["WD_Littbank_merge_Outer"].value_counts()



both          3187
left_only      532
right_only     167
Name: WD_Littbank_merge_Outer, dtype: int64

In [25]:
WDLittbank_WD_merge_Not_connected = \
WDLittbank_WD_merge[WDLittbank_WD_merge["WD_Littbank_merge_Outer"] == "left_only"].copy()

In [26]:
WDLittbank_WD_merge_Not_connected

,authorid,authorid_norm,db_checksum,db_timestamp,doc_type,full_name,gender,imported,intro,name_for_index,pictureinfo,searchable,show,surname,updated,birth.date,birth.plain,death.date,death.plain,librisid,wikidata.birthplace,wikidata.birthplace_label,wikidata.deathplace,wikidata.deathplace_label,wikidata.image,wikidata.sbl_link,wikidata.skbl_link,wikidata.sol_link,wikidata.wikidata_id,wikidata.wikipedia,db_timestamp_updated,intro_text,popularity,pseudonym,dramawebben.intro,dramawebben.intro_author,dramawebben.intro_author_norm,dramawebben.legacy_url,dramawebben.picture,sources,other_name,intro_author,intro_author_norm,dramawebben.picture_info,picture,bibliography,external_ref,presentation,seemore,dramawebben.sources,WikidataID,SBL,SKBL,WD_Littbank_merge_Outer
7,Addäos,Addaos,da5e35f5f2e32602a17344e44b2b6e93,1.675080e+12,author,Addäos,male,2014-08-22,None,Addäos,None,False,True,Addäos,2016-04-12,-350,0000,NaN,0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
30,AhlbergL,AhlbergL,ee16e9ff206c29114ebb85c687f26a3d,1.675080e+12,author,Ludvig Ahlberg,male,2021-03-01,None,"Ahlberg, Ludvig",None,True,True,Ahlberg,2021-03-01,NaN,1809,1888,1888,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.677767e+12,NaN,67.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
39,AhlmanC,AhlmanC,ded5b5ffeea552a3c1ea1778d4430364,1.675080e+12,author,Carl O. Ahlman,male,2019-02-25,None,"Ahlman, Carl O.",None,False,True,Ahlman,2019-02-25,1830,0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.675080e+12,NaN,290.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
48,AhrenbergB,AhrenbergB,b1741fb810e253a036639d61e7a3563f,1.675080e+12,author,Bertha Ahrenberg,female,2022-01-31,None,"Ahrenberg, Bertha",None,True,True,Ahrenberg,2022-01-31,1851,0000,NaN,0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.677767e+12,NaN,804.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
69,AlmquistCA,AlmquistCA,296bc1e66313d7f4d038ac24ebc1ef39,1.675080e+12,author,C. A. Almquist,not known,2021-03-01,None,"Almquist, C. A.",None,True,True,Almquist,2021-03-01,1825,0000,NaN,0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.677767e+12,NaN,382.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3684,ÅkerlundPA,AkerlundPA,305970795ebaa9fc00e54d3889a241f1,1.675080e+12,author,P. A. Åkerlund,not known,2021-04-06,None,"Åkerlund, P. A.",None,True,True,Åkerlund,2021-04-06,1843,0000,NaN,0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.677767e+12,NaN,125.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3688,ÅlundP,AlundP,7e20f1ee10e297c8935b8413bff60610,1.675080e+12,author,Paulus Ålund,male,2019-02-25,None,"Ålund, Paulus",None,True,True,Ålund,2019-02-25,1820,0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.677767e+12,NaN,541.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3707,OhmanFA,OhmanFA,eed1bf5666f0b20a897d2dd6ac37270c,1.675080e+12,author,Frans August Öhman,male,NaN,None,"Öhman, Frans August",None,False,True,Öhman,NaN,1821,1821,1892,1892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3714,ÖstergrenPJ,OstergrenPJ,27ee5cfe5cf3d88be34963b41f68331f,1.675080e+12,author,P. J. Östergren,male,2021-11-29,None,"Östergren, P. J.",None,True,True,Östergren,2021-11-29,1870,0000,NaN,0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.677767e+12,NaN,116.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [29]:
WDLittbank_WD_merge_Not_connected.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 532 entries, 7 to 3716
Data columns (total 54 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   authorid                       532 non-null    object  
 1   authorid_norm                  532 non-null    object  
 2   db_checksum                    532 non-null    object  
 3   db_timestamp                   532 non-null    float64 
 4   doc_type                       532 non-null    object  
 5   full_name                      532 non-null    object  
 6   gender                         532 non-null    object  
 7   imported                       371 non-null    object  
 8   intro                          30 non-null     object  
 9   name_for_index                 532 non-null    object  
 10  pictureinfo                    1 non-null      object  
 11  searchable                     532 non-null    object  
 12  show                           532 

In [30]:
WDLittbank_WD_merge_Not_connected["librisid"].value_counts()

hftwzq01397wbbd    1
0xbdfjvj5rjnrc0    1
20dgkckl01lmd80    1
Name: librisid, dtype: int64

In [33]:
WDLittbank_WD_merge_Not_connected[WDLittbank_WD_merge_Not_connected["librisid"] =="20dgkckl01lmd80"]

,authorid,authorid_norm,db_checksum,db_timestamp,doc_type,full_name,gender,imported,intro,name_for_index,pictureinfo,searchable,show,surname,updated,birth.date,birth.plain,death.date,death.plain,librisid,wikidata.birthplace,wikidata.birthplace_label,wikidata.deathplace,wikidata.deathplace_label,wikidata.image,wikidata.sbl_link,wikidata.skbl_link,wikidata.sol_link,wikidata.wikidata_id,wikidata.wikipedia,db_timestamp_updated,intro_text,popularity,pseudonym,dramawebben.intro,dramawebben.intro_author,dramawebben.intro_author_norm,dramawebben.legacy_url,dramawebben.picture,sources,other_name,intro_author,intro_author_norm,dramawebben.picture_info,picture,bibliography,external_ref,presentation,seemore,dramawebben.sources,WikidataID,SBL,SKBL,WD_Littbank_merge_Outer
2318,MöllerD,MollerD,feedf6c76fa233da9cd0d41e2c71482c,1.675080e+12,author,Daniel Möller,male,2018-01-18,None,"Möller, Daniel",None,True,True,Möller,2022-11-28,1974,1974,NaN,NaN,20dgkckl01lmd80,None,None,None,None,None,None,None,None,Q6000899,https://sv.wikipedia.org/wiki/Daniel_M%C3%B6ller,1.675080e+12,NaN,102.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [34]:
WDLittbank_WD_merge_Not_connected["wikidata.sbl_link"].value_counts()

18794    1
Name: wikidata.sbl_link, dtype: int64

In [41]:
#WDLittbank_WD_merge_Not_connected["sources"].value_counts()

In [40]:
#WDLittbank_WD_merge_Not_connected["popularity"].value_counts()

In [39]:
# WDLittbank_WD_merge_Not_connected["intro_text"].value_counts()